In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import JointState
import math
import time
# 조인트로 보내는 코드 : 굳
class JointStatePublisher(Node):
    def __init__(self):
        super().__init__('joint_state_publisher')
        self.publisher_ = self.create_publisher(JointState, 'joint_states', 10)
        self.joint_names = [
            "link2_to_link1",
            "link3_to_link2",
            "link4_to_link3",
            "link5_to_link4",
            "link6_to_link5",
            "link6output_to_link6"
        ]
    def publish_joint_positions(self, positions_rad):
        msg = JointState()
        msg.header.stamp = self.get_clock().now().to_msg()
        msg.name = self.joint_names
        msg.position = positions_rad
        # velocity, effort는 필요에 따라 추가 가능
        self.publisher_.publish(msg)
        self.get_logger().info(f"Published joint positions: {positions_rad}")
def main(args=None):
    rclpy.init(args=args)
    node = JointStatePublisher()
    # 예시: 6개 관절 각도를 라디안으로 지정
    goal_positions = [math.radians(0), math.radians(0), math.radians(0), 0.0, math.radians(0), 0.0]
    try:
        while rclpy.ok():
            node.publish_joint_positions(goal_positions)
            time.sleep(0.5)  # 0.5초마다 목표 관절값 퍼블리시
    except KeyboardInterrupt:
        pass
    node.destroy_node()
    rclpy.shutdown()
if __name__ == '__main__':
    main()

In [11]:
from ikpy.chain import Chain
import numpy as np

# URDF 파일 경로
urdf_path = "/home/jetcobot/colcon_ws/src/jetcobot/jetcobot_description/urdf/mycobot_280_m5/mycobot_280m5_with_gripper_parallel.urdf"

# base_elements를 실제 루트 링크 이름으로 지정
my_chain = Chain.from_urdf_file(
    urdf_path,
    base_elements=["g_base"]  # 루트 링크 이름을 URDF에 맞게 지정
)

# 목표 위치/자세 (4x4 변환행렬, 예시)
target_frame = np.eye(4)
target_frame[:3, 3] = [0.25, -0.10, 0.30]  # x, y, z 좌표 (미터 단위)

# 회전(자세)까지 지정하려면 아래와 같이 RPY → 회전행렬 변환 후 적용
import transforms3d
rpy = [0, 1.57, 0]  # 예시: Y축 기준 90도 회전
rot = transforms3d.euler.euler2mat(*rpy)
target_frame[:3, :3] = rot

# IK 계산
angles = my_chain.inverse_kinematics(target_frame)
print("Joint angles (rad):", angles)


ValueError: could not broadcast input array from shape (4,4) into shape (3,)

In [9]:
import numpy as np

# 목표 위치
target_position = [0.25, -0.10, 0.30]

# 목표 자세: Y축 90도 회전 행렬 (3x3)
c, s = np.cos(1.57), np.sin(1.57)
rot = np.array([
    [c, 0, s],
    [0, 1, 0],
    [-s, 0, c]
])

# IKPy에 올바르게 입력
angles = my_chain.inverse_kinematics(
    target_position,
    target_orientation=rot
)
print("Joint angles (rad):", angles)


Joint angles (rad): [ 0.          1.18920435  0.09699977 -0.46038608 -0.45378746  0.09625048
 -0.19187801  0.41170164  0.          0.        ]


In [10]:
angles

array([ 0.        ,  1.18920435,  0.09699977, -0.46038608, -0.45378746,
        0.09625048, -0.19187801,  0.41170164,  0.        ,  0.        ])

In [8]:
target_frame

array([[ 7.96326711e-04,  0.00000000e+00,  9.99999683e-01,
         2.50000000e-01],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        -1.00000000e-01],
       [-9.99999683e-01,  0.00000000e+00,  7.96326711e-04,
         3.00000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [4]:
# !pip install ikpy --break-system-packages

In [12]:
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Pose
from sensor_msgs.msg import JointState
from ikpy.chain import Chain
import numpy as np
import time
import transforms3d

In [13]:
quat = [0.7071, 0, 0.7071, 0]  # 예시 쿼터니언 (w, x, y, z)
transforms3d.quaternions.quat2mat(quat)

array([[ 1.11022302e-16,  0.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.00000000e+00,  0.00000000e+00,  1.11022302e-16]])